# Main predictor: Coffee shops
1. Read data
2. Extract licenses with coffee or cafe in them
3. Extract year they were licensed
4. Convert area to Chicago Community Area

In [8]:
import pandas as pd
import numpy as np
import os
os.chdir('/Users/sarwaridas/Desktop/IDS 702/Final Project/ids720_CoffeeAndCrimes/00_sourcedata')
col_list = [
    "ID",
    "LEGAL NAME",
    "DOING BUSINESS AS NAME",
    "ADDRESS",
    "CITY",
    "STATE",
    "ZIP CODE",
    "WARD",
    "PRECINCT",
    "WARD PRECINCT",
    "LICENSE STATUS",
    "LICENSE STATUS CHANGE DATE",
    "LICENSE DESCRIPTION",
    "BUSINESS ACTIVITY ID",
    "BUSINESS ACTIVITY",
    "LICENSE TERM START DATE",
    "LICENSE TERM EXPIRATION DATE",
    "DATE ISSUED",
    "LATITUDE",
    "LONGITUDE",
    'LOCATION'
]
df_full = pd.read_csv("/Users/sarwaridas/Desktop/Business_Licenses.csv",
    usecols=col_list,
    low_memory=False,
)

# https://data.cityofchicago.org/Community-Economic-Development/Business-Licenses/r5kz-chrr


## Cleaning for coffee shops and extracting year

In [9]:
df_full = df_full.loc[
    (df_full["STATE"] == "IL") & (df_full["CITY"] == "CHICAGO")
]  # subsetting for chicago
names_doingbusiness = [str(x).lower() for x in df_full["DOING BUSINESS AS NAME"]]
names_businessactivity = [str(x).lower() for x in df_full["BUSINESS ACTIVITY"]]
names_legalactivity = [str(x).lower() for x in df_full["LEGAL NAME"]]

keep_row = []
for a, b, c in zip(names_doingbusiness, names_businessactivity, names_legalactivity):
    keep = 0
    if (
        ("coffee" in a)
        or ("coffee" in b)
        or ("coffee" in c)
        or ("cafe" in a)
        or ("cafe" in b)
        or ("coffee" in c)
    ):
        keep = 1
    keep_row.append(keep)


df_full.insert(loc=0, column="names_doingbusiness", value=names_doingbusiness)
df_full.insert(loc=1, column="names_businessactivity", value=names_businessactivity)
df_full.insert(loc=2, column="names_legalactivity", value=names_legalactivity)
df_full.insert(loc=3, column="keep_row", value=keep_row)
df = df_full.loc[df_full.keep_row == 1]
df = df.drop(["LEGAL NAME", "DOING BUSINESS AS NAME", "CITY", "STATE"], axis=1)


In [10]:
df["lc_start_date"] = pd.to_datetime(df["LICENSE TERM START DATE"], format="%m/%d/%Y")
df['year']=df['lc_start_date'].map(lambda x: x.year)
#df.insert(loc=5, column='years', value=years)
df=df.loc[df.year>2000]
df = df.drop(
    [
        "LICENSE TERM START DATE",
        "LICENSE TERM EXPIRATION DATE",
        "DATE ISSUED",
        "LICENSE STATUS",
        "LICENSE STATUS CHANGE DATE",
    ],
    axis=1,
)
df.head()


,names_doingbusiness,names_businessactivity,names_legalactivity,keep_row,ID,ADDRESS,ZIP CODE,WARD,PRECINCT,WARD PRECINCT,LICENSE DESCRIPTION,BUSINESS ACTIVITY ID,BUSINESS ACTIVITY,LATITUDE,LONGITUDE,LOCATION,lc_start_date,year
201,arturo express,retail sales of perishable foods,arturo coffee inc,1,53931-20200916,130 S CANAL ST,60606,42.0,12.0,42-12,Retail Food Establishment,775,Retail Sales of Perishable Foods,41.879616,-87.639765,"(41.87961626801739, -87.63976452343368)",2020-09-16,2020.0
840,social junkie,"preparation of food, coffee or drinks",3848 n. harlem tavern llc,1,2551384-20171214,3848 N HARLEM AVE 1,60634,38.0,3.0,38-3,Retail Food Establishment,736,"Preparation of Food, Coffee or Drinks",41.950348,-87.807329,"(41.95034769269968, -87.80732944580947)",2017-12-14,2017.0
1223,social junkie,"preparation of food, coffee or drinks",3848 n. harlem tavern llc,1,2551384-20191216,3848 N HARLEM AVE 1,60634,38.0,3.0,38-3,Retail Food Establishment,736,"Preparation of Food, Coffee or Drinks",41.950348,-87.807329,"(41.95034769269968, -87.80732944580947)",2019-12-16,2019.0
2027,lutz continental cafe & pastry shop,retail sales of perishable foods,"pastry partners, inc.",1,1356917-20201216,2454-2458 W MONTROSE AVE,60618,47.0,21.0,47-21,Retail Food Establishment,775,Retail Sales of Perishable Foods,41.961409,-87.690796,"(41.96140869447139, -87.6907962429884)",2020-12-16,2020.0
2251,gaslight coffee roasters,retail sales of perishable foods,gaslight coffee roasters llc,1,2185009-20201016,2385 N MILWAUKEE AVE,60647,1.0,10.0,1-10,Retail Food Establishment,775,Retail Sales of Perishable Foods,41.924668,-87.699992,"(41.92466824748827, -87.69999236109766)",2020-10-16,2020.0


In [11]:
df.drop(['names_businessactivity', 'names_legalactivity','ADDRESS', 'ZIP CODE', 'WARD', 'PRECINCT',
       'WARD PRECINCT', 'LICENSE DESCRIPTION', 'BUSINESS ACTIVITY ID',
       'BUSINESS ACTIVITY','lc_start_date'],axis=1,inplace=True)

In [10]:
df.to_csv("CoffeeShops_byLatitude.csv")

# Mapping to Community Areas

#### Option A: This code demonstrates how to take the shapefile provided on the City of Chicago Open Data Portal and converts it to a JSON file, with top-level keys named for each neighborhood, and associated values that give the latitude and longitude pairs corresponding to the neighborhood's boundaries. For example: (https://github.com/craigmbooth/chicago_neighborhood_finder)

#Fails to build GDAL

#### Option B: Determining Chicago neighborhood from latitude and longitude. The script uses a GeoJSON file containing polygons of Chicago neighborhoods. Once all of the polygons have been generated, a simple point in polygon algorithm is used to determine which neighborhood contains the point of interest. (https://github.com/jkgiesler/parse-chicago-neighborhoods/blob/master/gps_to_neighborhood.py)

In [11]:
df_loc=pd.DataFrame(df.LONGITUDE)
df_loc['LATITUDE']=pd.DataFrame(df.LATITUDE)

In [12]:
loc = pd.read_csv("Locations_withCCA.csv")
lookup= pd.read_csv("location_lookup.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Locations_withCCA.csv'

In [ ]:
loc.CCA=loc.CCA.str.title()

In [ ]:
loc=loc.replace({'Andersonville': 'Edgewater','Archer Heights,West Elsdon':'Archer Heights','Armour Square,Chinatown':'Armour Square',
'Belmont Cragin,Hermosa':'Belmont Cragin','Avalon Park,Calumet Heights':'Avalon Park','Belmont Craigin,Hermosa':'Belmont Cragin',
'Brighton Park,Mckinley Park':'Brighton Park','Wicker Park,West Town':'West Town','Back Of The Yards':'New City','Bronzeville':'Douglas','Bucktown':'Logan Square',
'Chatham,Burnside':'Burnside', 'Garfield Park': 'East Garfield Park','Wrigleyville':'Lakeview','West Loop':'Near West Side','Washington Heights,Roseland':'Washington Heights',
'United Center':'Near West Side','Ukrainian Village And East Village':'West Town','Streeterville': 'Near North Side', 'Southeast Side':'South Chicago',
'South Shore, Grand Crossing':'South Shore','Sheffield & Depaul':'Lincoln Park', 'Sauganash,Forest Glen': 'Forest Glen', 'Rush & Division':'Near North Side',  'River North': 'Near North Side',
 'Printers Row':'Loop', 'Old Town': 'Near North Side', 'North Park,Albany Park':'Albany Park', 'Museum Campus':'Loop', 'Mount Greenwood,Morgan Park': 'Morgan Park',
  'Gold Coast':'Lincoln Park',
 'Grant Park':'Loop','Greektown':'Near West Side', 'Irving Park,Avondale': 'Avondale', 'Kenwood,Oakland': 'Kenwood','Jackson Park':'South Deering','Little Italy, Uic':'Near West Side','Marquette Park,Gage Park':'Gage Park',
 'Montclare, Galewood':'Montclare','Midway Airport':'Garfield Ridge','Little Village':'South Lawndale','Lakeview':'Lake View','Millenium Park':'Loop'
})


In [310]:
# df_loc.to_csv("Locations.csv")
## run gps_to_neighborhood.py
locations = loc
locations['LOCATION'] = list(zip(locations.LATITUDE, locations.LONGITUDE))
locations = locations.drop(["Unnamed: 0", "Unnamed: 0.1"], axis=1)
locations.head()


,LONGITUDE,LATITUDE,CCA,LOCATION
0,-87.639765,41.879616,Near West Side,"(41.879616268, -87.639764523)"
1,-87.807329,41.950348,Dunning,"(41.950347693, -87.807329446)"
2,-87.807329,41.950348,Dunning,"(41.950347693, -87.807329446)"
3,-87.690796,41.961409,Lincoln Square,"(41.961408694, -87.690796243)"
4,-87.699992,41.924668,Logan Square,"(41.924668247, -87.699992361)"


In [311]:
locations['CCA']= np.where(locations.LATITUDE.isnull(),np.nan, locations['CCA'])

In [312]:
#(100 * locations.isna().sum()) / len(locations)  # Missing 8.14% data for Wards


In [313]:
lookup.columns=['CCA_id','CCA']

In [314]:
locations_new=pd.merge(locations,lookup,on="CCA",how="left")
locations_new.CCA.nunique()

56

In [315]:
locations_new.shape

(15499, 5)

In [316]:
locations.shape

(15499, 4)

In [317]:
df.shape

(15499, 18)

In [318]:
df['CCA']=pd.DataFrame(locations['CCA'].values)
df=df.reset_index()

In [319]:
df['CCA']=pd.DataFrame(locations['CCA'].values)

In [320]:
(100 * df.isna().sum()) / len(df)  # Missing 8.14% data for Wards


index                      0.000000
names_doingbusiness        0.000000
names_businessactivity     0.000000
names_legalactivity        0.000000
keep_row                   0.000000
ID                         0.000000
ADDRESS                    0.000000
ZIP CODE                   0.038712
WARD                       0.019356
PRECINCT                   7.290793
WARD PRECINCT              0.019356
LICENSE DESCRIPTION        0.000000
BUSINESS ACTIVITY ID      11.355571
BUSINESS ACTIVITY         11.355571
LATITUDE                   0.625847
LONGITUDE                  0.625847
LOCATION                   0.625847
lc_start_date              0.000000
year                       0.000000
CCA                        4.606749
dtype: float64

In [321]:
coffee_by_year_CCA=df[["year","CCA","keep_row"]].groupby(["CCA","year"],as_index=False).count()

In [4]:
import pandas as pd
os.chdir('/Users/sarwaridas/Desktop/IDS 702/Final Project/ids720_CoffeeAndCrimes/02_processed_data')
#coffee_by_year_CCA.to_csv("coffee_by_year_CCA.csv")
coffee_by_year_CCA=pd.read_csv("coffee_by_year_CCA.csv")

In [5]:
coffee_by_year_CCA.head()

,Unnamed: 0,CCA,year,keep_row
0,0,Albany Park,2001.0,2
1,1,Albany Park,2002.0,22
2,2,Albany Park,2003.0,28
3,3,Albany Park,2004.0,32
4,4,Albany Park,2005.0,34


In [16]:
coffee_by_year_CCA.CCA.nunique()

56

In [324]:
coffee_by_year_CCA.CCA.unique()

array(['Albany Park', 'Archer Heights', 'Armour Square', 'Ashburn',
       'Auburn Gresham', 'Austin', 'Avalon Park', 'Avondale',
       'Belmont Cragin', 'Beverly', 'Bridgeport', 'Brighton Park',
       'Burnside', 'Douglas', 'Dunning', 'East Garfield Park',
       'Edgewater', 'Edison Park', 'Englewood', 'Forest Glen',
       'Gage Park', 'Garfield Ridge', 'Humboldt Park', 'Hyde Park',
       'Jefferson Park', 'Kenwood', 'Lake View', 'Lincoln Park',
       'Lincoln Square', 'Logan Square', 'Loop', 'Lower West Side',
       'Montclare', 'Morgan Park', 'Near North Side', 'Near South Side',
       'Near West Side', 'New City', 'North Center', 'North Lawndale',
       'Norwood Park', 'Portage Park', 'Pullman', 'Rogers Park',
       'South Chicago', 'South Deering', 'South Lawndale', 'South Shore',
       'Uptown', 'Washington Heights', 'Washington Park', 'West Lawn',
       'West Pullman', 'West Ridge', 'West Town', 'Woodlawn'],
      dtype=object)